<img src="https://lh3.googleusercontent.com/pw/ACtC-3fFHZrzKpHGWl0vYz7Sr8FX8QqLQ_tc8XHBSwqQnM4hgsIOjtjaOde1M9oHSAfe1Fs2SwVORlapit4-JOz0mjP8Tnz6HetkLZDZb8CifSd0uoSp1Nj3wG_wh1sEQlKXXzvEA9Y9HnQqu2Ecv2igmInb=w1097-h235-no?authuser=0" alt="2020年度ゲノム情報解析入門" height="100px" align="middle">

# 大規模データ解析入門 - MutMap解析 -

　今回は、「Pythonライブラリを使う」で学んだ2つのライブラリ（**pandas**、**Matolotlib**）の実践編です。それらのライブラリを使って、「MutMap解析」と呼ばれるゲノムデータ解析をおこないます。

### MutMap解析とサンプルデータの概要

　この実習では、イネの葉が「淡緑色（pale green）」となる形質の**MutMap解析** (Abe et al., 2012) をおこないます。

<img src="https://lh3.googleusercontent.com/pw/ACtC-3c1W_Ul3wYElrGq35UuP6Otsy9zce5kpT1BXDxpWXug3fWOC5BtOT_fmyok5uakfwbtlGzFlYjH-gopl8YNNVa62uUzimtc4_OdNzcSwb1kVbqW7hcST2MsgMs2Qe14E9qIWH58gglNIgxL0WurxOZ-=w395-h216-no?authuser=0" alt="mutmap04" height="200px">

　MutMap解析とは、形質に関わる遺伝子がゲノム上のどこにあるかを調べる解析手法（遺伝子マッピング手法）の一つです。次のような手順で解析を行います。

1. オリジナル系統に突然変異を誘発させて、突然変異系統を作成します。
1. 突然変異系統とオリジナル系統を交配します。
1. 交配後第二世代（F2集団）を作成し、突然変異形質を持つ個体のDNAを集めます（以下バルクDNA）。
1. オリジナル系統のDNAとバルクDNAをシーケンスします。
1. 両者のシーケンスを比較して、次のことを調べます。
  - 突然変異が誘発された位置
  - オリジナル系統の塩基（リファレンス塩基）と突然変異の塩基の種類
  - バルクDNAシーケンス中のリファレンス塩基と突然変異塩基の個数
1. 突然変異形質を持つF2個体は、ゲノム中のどこかの突然変異が共通しているはずです。突然変異形質F2個体のバルクDNAシーケンスを調べて、突然変異塩基のみ検出されるゲノム領域を探せば、形質の原因となる遺伝子領域を特定できます。MutMap解析では、次のことを調べます。
  - 原因遺伝子領域を探す指標として、リードデプス中の突然変異塩基の割合（**SNP-index**）を各SNP座位について調べます。
  - SNP-indexの推移をグラフに描き、形質の原因となる遺伝子領域を探します。

<img src="https://lh3.googleusercontent.com/pw/ACtC-3e5jUBTKSMRm9RM3cY2CsuHvlm0elGRVN6wdLgJ5qtOwUgknoazlf6Q2OxEhQHbGLj2eIBaxg5qa5He5vzldR2eW85M0eeZ2CjEVYoPTtbwJ7nJRsNRQEnBYYMxUaXyNyTESHezfQNYuIdhKMvDQttp=w720-h405-no?authuser=0" alt="mutmap_overview" height="400px">

<img src="https://lh3.googleusercontent.com/pw/ACtC-3dwfIF5-NArxJ_psrsuVdkVYg9uV-rxFuqVNcRC4zRaD6C5wOsma7f5i0X97cXti3Xw2wAW6XxgDks1zb2G5MmA7LXMwhoHCuw_vurtfgT6h964awbmQ9Y8tGVaxP4CkEYbQhcShzVu6DYKjtZ8FE9c=w1379-h303-no?authuser=0" alt="mutmap_overview_graph" height="200px">

　詳しく知りたい方は、下記の論文をご覧ください。

> Abe, A., Kosugi, S., Yoshida, K., Natsume, S., Takagi, H., Kanzaki, H., Matsumura, H., Yoshida, K., Mitsuoka, C., Tamiru, M., Innan, H., Cano, L., Kamoun, S., Terauchi, R. (2012). [Genome sequencing reveals agronomically important loci in rice using MutMap.](https://www.nature.com/articles/nbt.2095)  Nature biotechnology, 30(2), 174.


### サンプルデータのダウンロード


In [ ]:
"""
＊重要＊
最初にこのセルを実行してください。
この実習で使うサンプルファイルをダウンロードします。
"""
!wget -O mutmap_dataset.txt https://raw.githubusercontent.com/CropEvol/lecture/master/data/mutmap_chr10.txt

　[サンプルデータ](https://raw.githubusercontent.com/CropEvol/lecture/master/data/mutmap_chr10.txt)は、イネ第10染色体のSNP情報が書かれた **カンマ区切りファイル** です。

<img src="https://lh3.googleusercontent.com/pw/ACtC-3fNAxD422jnLtAXgJmheGNYjtxVoPwAAu3vGm27NfFEY035bmcdkESX2NlLBV-C3DmJNnwrrHprhXQex6B0GTNTMFDtHXEzMlqX-p1HyOE7atXWrof0DUpmgopj2-zSFBWf6v8dQDOmEZMEO2YU9dNc=w1147-h772-no?authuser=0" alt="mutmap_dataset" height="300px">



## 今回の実習内容

1. pandasでデータセットを読み込む
1. SNP-indexの計算に使用する列データを取り出す
1. SNP-indexを計算する
1. SNP-indexのグラフを描く
1. SNP-indexが0.8より大きいデータでプロットする

<img src="https://lh3.googleusercontent.com/pw/ACtC-3ehpu7szvJCN7Pw1vz9pGzeNiTQ6LXN-uj6ZIP-_5BoAmtwUnauQLrQlKxU5RMhiUZC0HgK0nzb7WkX7mhTgzb1sSzJbIP17aTujTsdBSu7oERctOYBMgB0w0bWNvjNfSMR9akDBWtLl9UiGBj8TaUD=w1456-h997-no?authuser=0" alt="mutmap_analysis_today" height="400px">

---

## 1. pandasでデータセットを読み込む

　次の3つのコードを書いて、データセットを読み込みましょう。

記述するコード:
- pandasライブラリを準備する（記述済み）
- ファイルを読み込む
- 表示する（記述済み）

In [ ]:
# 読み込みファイル名
dataset = "mutmap_dataset.txt"

# pandasライブラリを準備する
import pandas as pd

# ファイルを読み込む
df = 

# 表示する
df

#### コード例

In [ ]:
# 読み込みファイル名
dataset = "mutmap_dataset.txt"

# pandasライブラリを準備する
import pandas as pd

# ファイルを読み込む
df = pd.read_csv(dataset, sep=',', header=0)

# 表示する
df

## 2. SNP-indexの計算に使用する列データを取り出す

　SNP-indexを計算するために、2つの列データ（`N_REF`と`N_ALT`）を使います。その2つの列データを取り出して、それぞれを変数`n_ref`と変数`n_alt`に代入しましょう。

記述するコード:
- `N_REF`を取り出す
- `N_ALT`を取り出す

<img src="https://lh3.googleusercontent.com/pw/ACtC-3djSvhV73AXaNSg31bQ6SwUacotf1yoPQkSV_MLNVOufByxbqpvgHQKFRpgos3m2RfAzHMdKILIhFdkFwqvYLbrdVEtWX6H8O_5MTeWAqSVhNovu2LLb_CWHR6jIhNCf_VzMhwzPqG4RYRJ_2LEY6Oh=w906-h520-no?authuser=0" alt="mutmap_analysis_extract_columns" height="200px">

In [ ]:
# N_REFを取り出す
n_ref = 

# N_ALTを取り出す
n_alt = 

#### コード例

In [ ]:
# N_REFを取り出す
n_ref = df["N_REF"]

# N_ALTを取り出す
n_alt = df["N_ALT"]

## 3. SNP-indexを計算する

　先ほど取り出した2つの列データを使い、SNP-indexの計算をおこないましょう。また、計算した結果を新しい列`SNP_index`として追加しましょう。


$$ SNP\_index = \frac{N\_ALT}{N\_REF + N\_ALT} $$



記述するコード:
- SNP-indexを計算する
- 新しい列`SNP_index`を追加する

<img src="https://lh3.googleusercontent.com/pw/ACtC-3fLDU9W359E4CADsEzXE-9C74sJcjr-EuBF1oY9CesvKIu7JGxVITgguCdivSSMrTh-odmYvEwDuTsCs0YU5bGiluNFGtApUtVYeyPjsCFW27BF0lCebf-xDTUpKUeMr86PFmTUPgsCy3BbGBF_mglU=w1425-h520-no?authuser=0" alt="mutmap_analysis_calc_snpindex" height="200px">

In [ ]:
# SNP-indexを計算する
snp_index = 

# 新しい列SNP_indexを追加する
df["SNP_index"] = 

# 確認
df

#### コード例

In [ ]:
# SNP-indexを計算する
snp_index = n_alt / (n_ref + n_alt)

# 新しい列SNP_indexを追加する
df["SNP_index"] = snp_index

# 確認
df

## 4. SNP-indexのグラフを描く

　染色体上のSNP-indexの情報をグラフ（散布図）にしてみましょう。
- x軸: 染色体上の位置（列データ:`POS`）
- y軸: SNP-index（列データ:`SNP_index`）

記述するコード:
- Matplotlibライブラリを準備する（記述済み）
- x軸座標、y軸座標のデータを準備する
- SNP-indexのグラフを描く
  -  散布図を描くためのコードを追記する
  - グラフタイトル、x軸ラベル、y軸ラベルのコードを有効にする

<img src="https://lh3.googleusercontent.com/pw/ACtC-3emOB9m3Jzeq_nQbBBdCsOI-PUdQKJ48ANVmRdOHaCvUh9L6JCIoi1UN16C1Zhi_yOo-jvOmKiAh0dd5u6im6Dj2ZRpwfGPxX8PeRSZbAJKc31ZDc55VvRTYWyCYHwrmPEpUvRy_Z7R9DFf7rT9GUvd=w1456-h997-no?authuser=0" alt="mutmap_analysis_draw_graph" height="400px">

In [ ]:
# Matplotlibライブラリを準備する
import matplotlib.pyplot as plt

# x軸座標、y軸座標のデータを準備する
x = 
y = 

# SNP-indexのグラフを描く
plt.figure(figsize=[12,4])    # グラフサイズの設定
# (散布図を描くコード)

#plt.title('SNP-index on chromosome 10', fontsize=18)  # グラフタイトル
#plt.xlabel('Position (x10Mb)', fontsize=12)  # x軸ラベル
#plt.ylabel('SNP-index', fontsize=12)           # y軸ラベル
plt.show()                    # グラフ表示

#### コード例

In [ ]:
# Matplotlibライブラリを準備する
import matplotlib.pyplot as plt

# x軸座標、y軸座標のデータを準備する
x = df["POS"]
y = df["SNP_index"]

# SNP-indexのグラフを描く
plt.figure(figsize=[12,4])    # グラフサイズの設定
# (散布図を描くコード)
plt.scatter(x, y)
plt.title('SNP-index on chromosome 10', fontsize=18)  # グラフタイトル
plt.xlabel('Position (x 10 Mb)', fontsize=12)  # x軸ラベル
plt.ylabel('SNP-index', fontsize=12)           # y軸ラベル
plt.show()                    # グラフ表示

## 5. SNP-indexが0.8より大きいデータでプロットする

　最後に、全SNP-indexの散布図のうち、「SNP-indexが0.8より大きいデータ」のみを赤色のプロットにしてみましょう。

記述するコード:
- 全SNP-indexのグラフを描く（「4. SNP-indexのグラフを描く」と同じコード; 記述済み）
- SNP-indexが0.8より大きいデータを取り出す
- 取り出したデータのSNP-indexのグラフを描く
  - 赤色の散布図を描くためのコードを追記する

<img src="https://lh3.googleusercontent.com/pw/ACtC-3ewVsSaHA4mAPiZxGruHVZcdmmlcdZaTfPKNLU6EOL1MfyMoFQjF7WZ85CiwJklcf2zpWmSA3LcZ2LdatGqgdmWO0XaprtdyiLi2wC-_RQNYElDZUE9dD1nWYXXBhaeJMP1t7J2NwM0iMVTJTbaW2Xl=w1662-h658-no?authuser=0" alt="mutmap_analysis_draw_graph2" height="300px">

In [ ]:
# 全SNP-indexのグラフを描く
x = df["POS"]         # x軸座標データ
y = df["SNP_index"]  # y軸座標データ

import matplotlib.pyplot as plt
plt.figure(figsize=[12,4])    # グラフサイズの設定
plt.scatter(x, y)             # 散布図
plt.title('SNP-index on chromosome 10', fontsize=18)  # グラフタイトル
plt.xlabel('Position (x 10 Mb)', fontsize=12)  # x軸ラベル
plt.ylabel('SNP-index', fontsize=12)           # y軸ラベル

# SNP-indexが0.8より大きいデータを取り出す
df2 = 

# 取り出したデータのSNP-indexのグラフを描く


# グラフ表示
plt.show()                    

#### コード例

In [ ]:
# 全SNP-indexのグラフを描く
x = df["POS"]         # x軸座標データ
y = df["SNP_index"]  # y軸座標データ

import matplotlib.pyplot as plt
plt.figure(figsize=[12,4])    # グラフサイズの設定
plt.scatter(x, y)             # 散布図
plt.title('SNP-index on chromosome 10', fontsize=18)  # グラフタイトル
plt.xlabel('Position (x 10 Mb)', fontsize=12)  # x軸ラベル
plt.ylabel('SNP-index', fontsize=12)           # y軸ラベル

# SNP-indexが0.8より大きいデータを取り出す
df2 = df[ df["SNP_index"]>0.8 ]

# 取り出したデータのSNP-indexのグラフを描く
x2 = df2["POS"]         # x軸座標データ
y2 = df2["SNP_index"]   # y軸座標データ
plt.scatter(x2, y2, color="red")  # 散布図

#  グラフ表示
plt.show()   
df

---
## まとめ

　今回、pandasとMatplotlibを使ったデータ解析の実践として、**MutMap解析** をおこないました。pandasを使って、テキストデータを読み込み、列同士の数値計算（SNP-indexの計算）をおこないました。また、Matplotlibで、データをグラフ化しました。

　今回扱ったデータは、272行の小規模なデータ（イネ第10染色体のみのデータ）でしたが、pandasやMatplotlibを使うと、もっと大規模なデータ（数万行、数10万行のデータ）も同じように解析できます。

　MutMap解析は、**SNP-index** という指標を使って、形質に関わる遺伝子のゲノム上の位置を調べる解析です。今回、SNP-indexが高い場所（SNP-index>0.8）が、第10染色体上に何箇所かあることがわかりました。しかし、どこに原因遺伝子があるのでしょうか？　今のままでははっきりしません。

　次回、**Sliding window解析**というのをおこない、原因遺伝子の位置の絞り込みをおこないます。

